###  *Goal*: Use social media posts to explore the appplication of text and natural language processing to see what might be learned from online interactions.

Specifically, we will retrieve, annotate, process, and interpret Twitter data on health-related issues such as depression.

--- 
References:
* [Mining Twitter Data with Python (Part 1: Collecting data)](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)
* The [Tweepy Python API for Twitter](http://www.tweepy.org/)

Required Software
* [Python 3](https://www.python.org)
* [NumPy](http://www.numpy.org) - for preparing data for plotting
* [Matplotlib](https://matplotlib.org) - plots and garphs
* [jsonpickle](https://jsonpickle.github.io) for storing tweets. 
---

In [58]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import time
import operator
from datetime import datetime

# Introduction

This module continues the Social Media Data Science module started in [Part 1](SocialMedia - Part 1.ipynb) covering the annotation of tweets. These lessons will continue in [Part 3](SocialMedia - Part 1.ipynb) as we move on to the use of Natural Language processing to analyze the tweets. 
  
Our case study will apply these topics to Twitter discussions of smoking and vaping. Although details of the tools used to access data and the format and content of the data may differ for various services, the strategies and procedures used to analyze the data will generalize to other tools.

## 0. Setup

Before we dig in, we must grab a bit of code from [Part 1](SocialMedia - Part 1.ipynb):

1. The Tweets class used to store the tweets.
2. Our twitter API Keys - be sure to copy the keys that you generated when you completed [Part 1](SocialMedia - Part 1.ipynb).
3. Configuration of our Twitter connection

In [59]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.contents={}
        self.contents['time']=datetime.now()
        self.contents['tweets']={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        self.contents['searchTerm']=term
        self.contents['time']=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt)
            time.sleep(5)
                
    def addTweet(self,tweet):
        id = tweet['id_str']
        tweets=self.contents['tweets']
        if id not in tweets.keys():
            tweets[id]={}
            tweets[id]['tweet']=tweet
            tweets[id]['count']=0
        # if a count is not provided in the call, increment the count 
        if count == 0:
            tweets[id]['count'] = tweets[id]['count'] +1
        else:
            tweets[id]['count'] = count
        
    def getTweet(self,id):
        id = str(id)
        tweets=self.contents['tweets']
        if id in tweets:
            return tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.contents['tweets'][id]['count']
    
    def countTweets(self):
        return len(self.contents['tweets'])
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        tweets=self.contents['tweets']
        for t,entry in tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.contents['tweets'].keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.contents)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        self.contents={}
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.contents=incontents
        
    def getSearchTerm(self):
        return self.contents['searchTerm']
    
    def getSearchTime(self):
        return self.contents['time']

*REDACT FOLLOWING DETAILS*

In [60]:
consumer_key='D2L4YZ2YrO1PMix7uKUK63b8H'
consumer_secret='losRw9T8zb6VT3TEJ9JHmmhAmn1GXKVj30dkiMv9vjhXuiWek9'
access_token='15283934-iggs1hiZAPI2o5sfHWMfjumTF7SvytHPjpPRGf3I6'
access_secret='bOvqssxS97PGPwXHQZxk83KtAcDyLhRLgdQaokCdVvwFi'

In [61]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## 1. Annotating Tweets

Now that we have a corpus of tweets, what do we want to do with them? Turning a relatively vague notion into a well-defined research question is often a significant challenge, as examination of the data often reveals both shortcomings and unforeseen opportunities.

In our case, we are interested in looking at tweets about depression, but we're not quite sure exactly *what* we are looking for. We have a vague notion that we might learn something interesting, but understanding exactly what that is, and what sort of analyses we might need, will require a bit more work.

In situations such as this, we might look at some of the data to form some preliminary impressions of the content. Specifically, we can look at indidividual tweets, assigning them to one or more categories - known as *codes* - based on their content.  We can add categories as needed to capture important ideas that we might want to refer back to. This practice - known as *open coding* allows us to begin to make sense of unfamiliar data sets. 

This sounds much more complicated than it is. For now, let's read some tweets in from a file collected using the procedures discused in [Part 1](SocialMedia - Part 1.ipynb). We'll then use those tweets to get to work. The tweets are stored in a file called `tweets-smoking.json`.

In [62]:
tweets =Tweets()
tweets.readTweets("tweets.json")

We check the count, search term, and date of collection to understand the origin of these tweets

In [65]:
print(tweets.countTweets())
print(tweets.getSearchTerm())
print(str(tweets.getSearchTime()))

100
smoking
2018-03-07 16:59:56.921697


We will begin by taking a look at a subset of the first 20 tweets

To get this list, we'll sort the ids of the tweets and take the first 10 in the list, as ordered by ID

In [66]:
ids=list(tweets.getIds())
ids.sort()
working=[]
for i in range(20):
    id = ids[i]
    working.append(id)

*working* now has 20 tweets ids. Let's start with the first.

In [67]:
td = working[0]
t = tweets.getTweet(td)
t['text']

'RT @NBCNews: Police: Teen tried to detonate bomb at Utah school; backpack with device was found smoking; teen had been researching about IS…'

This tweet has several interesting charcteristics.
1. it appears to be a retweet.
2. It contains a user mention 

We can model all of these points through relevant annotation. Specifically, we will a new array of codes to each tweet object. This array will contain a list of categorical annotations associated with the tweet.  We add routines to add a single code to a tweet (by ID), to add multiple codes, and to retrieve the list of codes associated with a tweet.


See modifications to the  Tweets object in this new definition. 

In [68]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.contents={}
        self.contents['time']=datetime.now()
        self.contents['tweets']={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        self.contents['searchTerm']=term
        self.contents['time']=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt)
            time.sleep(5)
                
    def addTweet(self,tweet):
        id = tweet['id_str']
        tweets=self.contents['tweets']
        if id not in tweets.keys():
            tweets[id]={}
            tweets[id]['tweet']=tweet
            tweets[id]['count']=0
        # if a count is not provided in the call, increment the count 
        if count == 0:
            tweets[id]['count'] = tweets[id]['count'] +1
        else:
            tweets[id]['count'] = count
        
    def getTweet(self,id):
        id = str(id)
        tweets=self.contents['tweets']
        if id in tweets:
            return tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.contents['tweets'][id]['count']
    
    def countTweets(self):
        return len(self.contents['tweets'])
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        tweets=self.contents['tweets']
        for t,entry in tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.contents['tweets'].keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.contents)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        self.contents={}
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.contents=incontents
        
    def getSearchTerm(self):
        return self.contents['searchTerm']
     
                
    ### NEW ROUTINE - add a code to a tweet
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
        
    ### NEW ROUTINE  - add multiple  codes for a tweet
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
    ### NEW ROUTINE get codes for a tweet
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']

Now that we have this set up, we can reload the tweets from the file and reload the subset.

In [69]:
tweets =Tweets()
tweets.readTweets("tweets-smoking.json")
ids=list(tweets.getIds())
ids.sort()
working=[]

for i in range(20):
    id = ids[i]
    working.append(id)

td = working[0]
t = tweets.getTweet(td)
t['text']

'RT @NBCNews: Police: Teen tried to detonate bomb at Utah school; backpack with device was found smoking; teen had been researching about IS…'

Above we noted that this tweet was interesting becuase:
1. it is a retweet
2. It contains a user mention

So, we will add codes to the appropriate tweet as needed:

In [70]:
tweets.addCode(td,"RETWEET")
tweets.addCode(td,"USERMENTION")

We can also confirm that this tweet is associated with the desired codes:

In [71]:
tweets.getCodes(td)

{'RETWEET', 'USERMENTION'}

We can see if this is a retweet by checking for the `retweeted_status` attribute

In [72]:
'retweeted_status' in t

True

Good. Let's look at the next tweet. 

In [73]:
td = working[1]
t=tweets.getTweet(td)
t['text']

'RT @kushplug20: @Budfessions Smoking marijuana lowers your body temperature and makes you cool, Dm for orders and questions. #marijuana,#Ma…'

This tweet is a retweet with a user mention and several hashtags. We might eventually be interested in stroing all hashtags, but for now, the most interesting aspect is likely that this tweet mentions marijuana. So, we add a code for `marijuana`.

This time, we wil, for simplicity, use the `addCodes` routine to add the codes.

In [74]:
tweets.addCodes(td,["RETWEET", 'USERMENTION','HASHTAG','MARIJUANA'])

In [75]:
'retweeted_status' in t

True

ok.. moving on to the third tweet..

In [76]:
td = working[2]
t=tweets.getTweet(td)
t['text']

"@nytimes Why?\nWe now have a statue of a crack smoking mayor in DC, what's the big deal?\n#LiberalismIsAMentalDisorder"

This tweet includes a user mention, a hashtag, and a mention of a specific drug (`crack`).

In [77]:
tweets.addCodes(td,['RETWEET','USERMENTION','HASHTAG','CRACK'])

In [78]:
'retweeted_status' in t

False

next...

In [79]:
td = working[3]
t=tweets.getTweet(td)
t['text']

'RT @theblaze: The homemade bomb failed to go off and was discovered in a common area at the school by a student who reported a smoking back…'

Note that this tweet talks about a `smoking back..` (probably a smoking backpack). As interesting as this might be in some sense, it is probably not relevant to discussions of inhaled smoke. Thus, we mark it as `IRRELEVANT`.

In [80]:
tweets.addCodes(td,['RETWEET','USERMENTION','IRRELEVANT'])

In [81]:
'retweeted_status' in t

True

In [82]:
td = working[4]
t=tweets.getTweet(td)
t['text']

'RT @RAETHED0N: i love smoking weed. i hate advice.'

This retweet includes a user mention, a reference to marijuana (`weed`), and a statement of positive affect (` i love smoking weeed.`).

In [83]:
tweets.addCodes(td,['RETWEET','USERMENTION','MARIJUANA','POSAFFECT'])

In [84]:
td = working[5]
t=tweets.getTweet(td)
t['text']

'RT @maciassalyssa: Smoking weed is a turn off\U0001f927don’t @ me'

A retweet with a user mention, a reference to marijuana, and a comment with negative affect.

In [85]:
tweets.addCodes(td,['RETWEET','USERMENTION','MARIJUANA','NEGAFFECT'])

In [86]:
td = working[6]
t=tweets.getTweet(td)
t['text']

'RT @usatodayvideo: An Indonesian zoo is taking heat after video surfaced of an orangutan smoking like a pro. https://t.co/d0fvCxzRn5'

In [87]:
tweets.addCodes(td,['RETWEET','LINK','USERMENTION'])

In [88]:
td = working[7]
t=tweets.getTweet(td)
t['text']

'RT @NBCNews: Police: Teen tried to detonate bomb at Utah school; backpack with device was found smoking; teen had been researching about IS…'

This tweet is similar to the above tweet  about the backpack.

In [89]:
tweets.addCodes(td,['RETWEET','USERMENTION','IRRELEVANT'])

In [90]:
td = working[8]
t=tweets.getTweet(td)
t['text']

"RT @mrsdianek: @FoxNews @realDonaldTrump Whatever you're smoking, send me some.  I need a break from reality too."

This looks irrelevant

In [92]:
tweets.addCodes(td,['RETWEET','USERMENTION','IRRELEVANT'])

Now that we've gone through several tweets, we can review the codes used.

In [93]:
for i in range(9):
    td=working[i]
    print(tweets.getCodes(td))

{'RETWEET', 'USERMENTION'}
{'RETWEET', 'MARIJUANA', 'USERMENTION', 'HASHTAG'}
{'RETWEET', 'CRACK', 'USERMENTION', 'HASHTAG'}
{'RETWEET', 'IRRELEVANT', 'USERMENTION'}
{'RETWEET', 'MARIJUANA', 'POSAFFECT', 'USERMENTION'}
{'RETWEET', 'MARIJUANA', 'USERMENTION', 'NEGAFFECT'}
{'RETWEET', 'LINK', 'USERMENTION'}
{'RETWEET', 'IRRELEVANT', 'USERMENTION'}
{'RETWEET', 'IRRELEVANT', 'USERMENTION'}


Having annotated several tweets, we might want to save the annotations in a file for future use. Fortnuately, the approach that we've used in our save and reload code is flexible enough to handle this without any further changes to the implementation. 

How does this work? The `Tweets` class stores all of the information abou the tweets in a simple dictionary. Tweet counts and codes are then stored inside the tweet object. When we go to save the set of Tweets, we simply turn this dictionary into JSON and then write it to a file. To read things in, we just read the JSON from the file and convert the result back into a dictionary. Thus, anything that we add to the dictionary will automatically be writen out and read back in.  We still need additional routines to access this data (like `addCode`, `addCodes`, and `getCodes`), but we  don't need to change the save/load routines.  Let's try it out.


In [162]:
tweets.saveTweets("tweets-smoking-annotated.json")

In [163]:
tweets2=Tweets()
tweets2.readTweets("tweets-smoking-annotated.json")

In [101]:
print(tweets.getTweet(td)['text'])
print(tweets.getCodes(td))
print(tweets2.getTweet(td)['text'])
print(tweets2.getCodes(td))

RT @mrsdianek: @FoxNews @realDonaldTrump Whatever you're smoking, send me some.  I need a break from reality too.
{'RETWEET', 'IRRELEVANT', 'USERMENTION'}
RT @mrsdianek: @FoxNews @realDonaldTrump Whatever you're smoking, send me some.  I need a break from reality too.
{'RETWEET', 'IRRELEVANT', 'USERMENTION'}


****
## Exercise 2.1: Examining the distribution of codes across a corpus

Having annotated a number of tweets, you might want to get an idea of how many tweets are being used and how often. Write a new method inside the `Tweets` class tp will provide a list of codes and counts, sorted by frequency.  Please be sure to reload the tweets after you redefine the class.

*ANSWER FOLLOWS - cut below here*
Following lines to be deleted when provided for student use

In [102]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.contents={}
        self.contents['time']=datetime.now()
        self.contents['tweets']={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        self.contents['searchTerm']=term
        self.contents['time']=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt)
            time.sleep(5)
                
    def addTweet(self,tweet):
        id = tweet['id_str']
        tweets=self.contents['tweets']
        if id not in tweets.keys():
            tweets[id]={}
            tweets[id]['tweet']=tweet
            tweets[id]['count']=0
        # if a count is not provided in the call, increment the count 
        if count == 0:
            tweets[id]['count'] = tweets[id]['count'] +1
        else:
            tweets[id]['count'] = count
        
    def getTweet(self,id):
        id = str(id)
        tweets=self.contents['tweets']
        if id in tweets:
            return tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.contents['tweets'][id]['count']
    
    def countTweets(self):
        return len(self.contents['tweets'])
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        tweets=self.contents['tweets']
        for t,entry in tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.contents['tweets'].keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.contents)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        self.contents={}
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.contents=incontents
        
    def getSearchTerm(self):
        return self.contents['searchTerm']
     
                
    ### NEW ROUTINE - add a code to a tweet
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
        
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        tweets=self.contents['tweets']
        for id in tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [160]:
tweets=Tweets()
tweets.readTweets("tweets-smoking-annotated.json")

In [161]:
tweets.getCodeProfile()

[('VAPING', 1),
 ('USERMENTION', 17),
 ('RETWEET', 16),
 ('POSAFFECT', 2),
 ('NEGAFFECT', 2),
 ('MARIJUANA', 5),
 ('LINK', 4),
 ('IRRELEVANT', 5),
 ('HASHTAG', 3),
 ('CRACK', 1)]

*END CUT HERE*
****

****

## Exercise 2.2: Code the Next 10 tweets in the set. 
Start with the tags used above, adding your own as needed.  Code up to and including the tweet  with index 20 in the `working` array. Examine the code profile and save your tweets  to a new file when you are done. 

*ANSWER FOLLOWS - cut below here*
Following lines to be deleted when provided for student use

In [136]:
ids=list(tweets.getIds())
ids.sort()
working=[]

for i in range(9,19):
    id = ids[i]
    working.append(id)

td = working[0]
t=tweets.getTweet(td)
t['text']

'RT @ohPudu: Rare colorized photo of Virgin Mary smoking and Jesus. https://t.co/Z9Pr6t4dBn'

In [137]:
tweets.addCodes(td,['RETWEET','USERMENTION','LINK'])

In [138]:
td = working[1]
t=tweets.getTweet(td)
t['text']

'RT @BuzzFeedNews: A teenage boy who allegedly spray painted a message about ISIS on a Southern Utah high school, has been arrested for leav…'

In [139]:
tweets.addCodes(td,['RETWEET','USERMENTION','IRRELEVANT'])

In [140]:
td = working[2]
t=tweets.getTweet(td)
t['text']

'RT @JWilltize: @RMadridBabe I think that smoke was coming from whatever Xavi was smoking'

In [141]:
tweets.addCodes(td,['RETWEET','USERMENTION'])

In [142]:
td = working[3]
t=tweets.getTweet(td)
t['text']

"RT @stoner_stuff: DON'T EVER APOLOGIZE for SMOKING WEED #WednesdayWisdom"

In [143]:
tweets.addCodes(td,['RETWEET','USERMENTION','MARIJUANA','POSAFFECT'])

In [144]:
td = working[4]
t=tweets.getTweet(td)
t['text']

'@Sporf What is he smoking?'

In [145]:
tweets.addCodes(td,['USERMENTION'])

In [146]:
td = working[5]
t=tweets.getTweet(td)
t['text']

'RT @shujaxhaider: the guy sitting next to me on the bus is watching a video of himself smoking weed'

In [147]:
tweets.addCodes(td,['RETWEET','USERMENTION','MARIJUANA'])

In [148]:
td = working[6]
t=tweets.getTweet(td)
t['text']

"Stop smoking, we love you\nAnd we don't want you to die\n#CarSeatHeadrest #StopSmoking https://t.co/mIQvVzAWTD"

In [149]:
tweets.addCodes(td,['HASHTAG','LINK','NEGAFFECT'])

In [150]:
td = working[7]
t=tweets.getTweet(td)
t['text']

'Using household cleaning products could be as bad for you as smoking https://t.co/UlYvTqJQZZ https://t.co/XKv6ahNUI6'

In [151]:
tweets.addCodes(td,['LINK'])

In [152]:
td = working[8]
t=tweets.getTweet(td)
t['text']

'RT @aubreyrutten: Vaping and smoking are SO unattractive, idk how people like that tbh'

In [153]:
tweets.addCodes(td,['RETWEET','USERMENTION','VAPING'])

In [154]:
td = working[9]
t=tweets.getTweet(td)
t['text']

'RT @AndrewCollins: I am sad, but not surprised, that the @NME has stopped the presses. I remain deeply proud of having been associated with…'

In [155]:
tweets.addCodes(td,['RETWEET','USERMENTION','IRRELEVANT'])

In [156]:
tweets.getCodeProfile()

[('VAPING', 1),
 ('USERMENTION', 17),
 ('RETWEET', 16),
 ('POSAFFECT', 2),
 ('NEGAFFECT', 2),
 ('MARIJUANA', 5),
 ('LINK', 4),
 ('IRRELEVANT', 5),
 ('HASHTAG', 3),
 ('CRACK', 1)]

In [157]:
tweets.saveTweets("tweets-smoking-annotated.json")

*END CUT*

---

---
## EXERCISE 2.3: Reflection on coding

Open coding can often be an iterative process. When we first start out, we don't really know what we're looking for. As a result, the first few items annotated might only get a few codes, and we might miss ideas that we don't initially think are important. As we see more and more items, our ideas of what needs to be annotated will change, and we'll start adding in codes that might also apply to earlier messages. Thus, we often need to review and re-annotate earlier tweets to account for changes in our interpreations.

Review your annotations the tweets that you reviewed. Revise the codes associated with these tweets, adding items from the overall list of codes as appropriate. Describe the change that you have made.


---
*ANSWER FOLLOWS - cut below here*
Following lines to be deleted when provided for student use


*END CUT*

---

## EXERCISE 2.4: Reflection on storage/serialization

In working with this small set of 100 tweets, we are taking a very simple approach to storage and management of the tweets and annotations. Storing everything in a nested Python dictionary and then dumping it to disk as JSON text can be very appealing. What are the strengths and weaknesses of this approach, and how might these strengths and weaknesses differ with larger datasets containing 100,000 or 100 million datasets? What alternative  strategies might you use for larger datasets?

---
*ANSWER FOLLOWS - cut below here*
Following lines to be deleted when provided for student use

*Advantages*: JSON is easy to read, as programmers can open the text file and read contents directly. JSON is also easy to work with from multiple programming languages and platforms, allowing a collection of tweets written in Python to be read by code written in other languages.  Finally, the JSON structure is easily adaptable, allowing fields to be easily added or removed as needed.

*Disadvantages*: As a text-based format, JSON data is pretty-much "all or nothing". Although it might be possible to read in part of the JSON structure, this will complicate code significantly.  Thus, most programs would read the entire JSON structure into data all at once. This is fine for small datasets, but might get slow and bulky, requiring lots of RAM, for larger datasets. 

Very large sets of tweets might be stored in a database. A relational database might be created to store tweets in one or more tables, providing the power of the structured query language (SQL) to retrieve tweets matching only specified criteria.  SQL could also be used to quickly and easily calculate aggregate statistics, without loading all of the tweets into RAM. A downside of this approach is the need to manage a distinct software component (the database server), the difficulty in changing the contents of the database, the complexity of SQL queries, and the relatively complex and often language-specific tools needed to manage the connections to the database. 

Alternatively, NoSQL databases that work very well with JSON might be considered. These databases may share some of the challenges associated with relational databases, but they are often also more flexible.


*END CUT*

---

### 4.2 Final Notes

Now that we have completed the initial annotation, you can move on to [Part 3](SocialMedia - Part 3.ipynb)